# Control Centre

In [1]:
import time
import serial
import numpy as np
import sys
import os
import asyncio
from multiprocessing import Process
import threading
from scipy import constants
import itertools

import binascii
from datetime import datetime
import logging

#from gsioc import gsioc_Protocol, ensure_xy_position_will_be_reached

## Get started with GSIOC

To reset the COM port:
1) Open device manager and disable COM1
2) Turn machine off and on
3) Enable COM1 in device manager

Use ser.close() here to close the serial port

NOTE: Python seems to have problems when the GX-241 is in COM1 (motherboard), it will connect but cannot send or recieve. This is easily fixed by using a USB to serial adapter instead.

In [2]:
from equip_class import *

In [3]:
from platform_setup_new import *

In [4]:
# Set up the serial connection
PORT1 = 'COM6'
ser = serial.Serial(PORT1, 19200, 8, "N", 1, 0.1)

In [5]:
# Create an instance of the gsioc_Protocol class
g = gsioc_Protocol(ser, 'GX-241 II', 30)

In [6]:
g.connect()

2025-02-12 14:25:09.691 | DEBUG    | equip_class:verify_open:216 - Check if port is open.
2025-02-12 14:25:10.204 | DEBUG    | equip_class:iCommand:285 - Sending immediate command % to device.
2025-02-12 14:25:12.209 | DEBUG    | equip_class:iCommand:314 - Sending immediate command complete.
2025-02-12 14:25:12.212 | DEBUG    | equip_class:iCommand:324 - received bytearray(b'GX-241 II v2.0.2.5') as response.
2025-02-12 14:25:12.415 | DEBUG    | equip_class:connect:268 - Connected to device 30


In [7]:
g.iCommand('%')

2025-02-12 14:25:14.827 | DEBUG    | equip_class:iCommand:285 - Sending immediate command % to device.
2025-02-12 14:25:16.811 | DEBUG    | equip_class:iCommand:314 - Sending immediate command complete.
2025-02-12 14:25:16.814 | DEBUG    | equip_class:iCommand:324 - received bytearray(b'GX-241 II v2.0.2.5') as response.


'GX-241 II v2.0.2.5'

In [ ]:
g.bCommand('e[n]')

In [ ]:
def g_inject():
    g.bCommand('X146.5/0')
    g.bCommand('Z105')

In [ ]:
# Create an instance of the gsioc_Protocol class
g_dim = gsioc_Protocol(ser, 'GX D Inject', 3)

In [ ]:
g_dim.connect()

In [ ]:
g_dim.iCommand('%')

In [ ]:
 g_dim.bCommand('VI')

In [ ]:
g_dim.bCommand('VL')

In [ ]:
def pos_change():
    if g_dim.iCommand('X') == 'L' :
        g_dim.bCommand('VI')
    else :
        g_dim.bCommand('VL')

In [ ]:
pos_change()

## Basic method byte by byte

In [ ]:
ser.isOpen()

In [ ]:
# 4. Testing and Debugging
# Create an instance of DeviceController
device = DeviceController(ser)

# Example command to send (replace with actual command for your device)
command_string = '%'

# Call the iCommand method
try:
    response = device.iCommand(command_string)
    print(f'Response from device: {response}')
except Exception as e:
    print(f'Error: {e}')

In [ ]:
ser.close()

## Try Run and Platform Setup

This seems a little complex as is, too many unneeded things bundled in run.py. Try to extract the relevant functions and start again.

In [8]:
rack_position_offset_x=92       #distance in mm between rack_position=1 and =2 on x-axis
rack_position_offset_y=0        #distance in mm between rack_position=1 and =2 on y-axis

############################# RACK 1 DEFINITION #################################

# From platform_setup.py 
rack1 = Rack([4,16], 7.5, 39.5, 18.5, 13.75, 65) # groundlevel_height assumed the minimum Z

#  array_dimensions, offset_x, offset_y=offset_y, vial2vial_x, vial2vial_y, groundlevel_height

# Previous vial2vialx = (2.11+15.6)
# Previous vial2vial7 = (2.72+15.6+0.35)

array_order1 = np.array([      #user is obliged to define a integer number i>=1 for each vial in the rack in ascending order 
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10,11,12],
    [13,14,15,16],
    [17,18,19,20],
    [21,22,23,24],     
    [25,26,27,28],     
    [29,30,31,32],
    [33,34,35,36],
    [37,38,39,40],
    [41,42,43,44],
    [45,46,47,48],
    [49,50,51,52],
    [53,54,55,56],
    [57,58,59,60],
    [61,62,63,64]        
    ])
    
rack_pos1=1

global rack1_commands

# Not sure what rack_position_offset_x/y are for x=92 and y=0

rack1_commands = Rackcommands(rack1, array_order1, rack_pos1, rack_position_offset_x, rack_position_offset_y)

global vial_selfmade

vial_selfmade = Vial(1.5, 1, 33, 31.08)

In [9]:
# It works!
thing = rack1_commands.get_xy_command(2)
thing
g.bCommand(thing[0])

2025-02-12 14:25:41.569 | DEBUG    | equip_class:bCommand:331 - Sending buffered command "X26.0/39.5" to device.
2025-02-12 14:25:41.570 | INFO     | equip_class:bCommand:335 - GSIOC <<< X26.0/39.5
2025-02-12 14:25:41.683 | DEBUG    | equip_class:bCommand:354 - ready signal: b'\n'
2025-02-12 14:25:41.685 | DEBUG    | equip_class:bCommand:363 - Starting to send buffered command.
2025-02-12 14:25:41.687 | DEBUG    | equip_class:bCommand:374 - 10
2025-02-12 14:25:41.689 | DEBUG    | equip_class:bCommand:379 - Writes buffered Command character b'X' to the device GX-241 II.
2025-02-12 14:25:41.691 | DEBUG    | equip_class:bCommand:381 - Starts reading character from the device GX-241 II.
2025-02-12 14:25:41.793 | DEBUG    | equip_class:bCommand:383 - got back b'X' from the device GX-241 II.
2025-02-12 14:25:41.793 | DEBUG    | equip_class:bCommand:387 - resp_raw: b'X'
2025-02-12 14:25:41.794 | DEBUG    | equip_class:bCommand:405 - resp_raw 2: 88
2025-02-12 14:25:41.795 | DEBUG    | equip_cl

bytearray(b'\nX26.0/39.5\r')

In [9]:
def go_to_vial(self, vial):
    rack1_commands.get_xy_command(2)
    self.bCommand(thing[0])
    log_action('test_log.txt', f"Autosampler sent to {vial} position.")

In [10]:
g.go_to_vial(3)

NameError: name 'rack1_commands' is not defined

In [10]:
g.bCommand('H')

2025-02-12 14:25:52.380 | DEBUG    | equip_class:bCommand:331 - Sending buffered command "H" to device.
2025-02-12 14:25:52.381 | INFO     | equip_class:bCommand:335 - GSIOC <<< H
2025-02-12 14:25:52.488 | DEBUG    | equip_class:bCommand:354 - ready signal: b'\n'
2025-02-12 14:25:52.489 | DEBUG    | equip_class:bCommand:363 - Starting to send buffered command.
2025-02-12 14:25:52.489 | DEBUG    | equip_class:bCommand:374 - 10
2025-02-12 14:25:52.490 | DEBUG    | equip_class:bCommand:379 - Writes buffered Command character b'H' to the device GX-241 II.
2025-02-12 14:25:52.490 | DEBUG    | equip_class:bCommand:381 - Starts reading character from the device GX-241 II.
2025-02-12 14:25:52.596 | DEBUG    | equip_class:bCommand:383 - got back b'H' from the device GX-241 II.
2025-02-12 14:25:52.599 | DEBUG    | equip_class:bCommand:387 - resp_raw: b'H'
2025-02-12 14:25:52.611 | DEBUG    | equip_class:bCommand:405 - resp_raw 2: 72
2025-02-12 14:25:52.621 | DEBUG    | equip_class:bCommand:379 -

bytearray(b'\nH\r')

## With syr pump

In [14]:
#Defining ser
ser_1 = serial.Serial()

In [15]:
# Create an instance of the gsioc_Protocol class
pump = SyrPump(ser_1, 'K100 V01.30', 1)

In [16]:
#Connecting to pump
pump.connect()

Device is connected


In [17]:
#clearing recorded volumes
pump.command('cvolume')
pump.command('ctvolume')

''

In [18]:
#setting target vol
pump.command('tvolume 1 ml')

''

In [ ]:
# It works!
thing = rack1_commands.get_xy_command(1)
thing
g.bCommand(thing[0])

In [ ]:
g.bCommand('Z85')

In [19]:
pump.command('wrate 1 ml/min')
pump.command('wran')

''

In [ ]:
# It works!
thing = rack1_commands.get_xy_command(2)
thing
g.bCommand(thing[0])

In [ ]:
g.bCommand('Z85')
sleep(5)

In [20]:
pump.command('irate 1 ml/min')
pump.command('irun')

''

In [ ]:
g.bCommand('H')

## SPKA code - to put in a function

In [ ]:
# Define kinetic screen
no_sub_A = 1      # First Column
no_sub_B = 0      # Second Column
no_cat = 0        # Third Column
no_add = 0        # Fourth Column

# Create lists for each
sub_A_list = [1 + 4 * i for i in range(no_sub_A)]
sub_B_list = [2 + 4 * i for i in range(no_sub_B)] if no_sub_B > 0 else []
cat_list = [3 + 4 * i for i in range(no_cat)] if no_cat > 0 else []
add_list = [4 + 4 * i for i in range(no_add)] if no_add > 0 else []

#print(sub_A_array, sub_B_array, cat_array)

In [ ]:
# Generate all possible combinations
all_combinations = list(itertools.product(sub_A_list, sub_B_list, cat_list, add_list))

# Reorder combinations so that all combinations with the same elements
# in sub_B_array and cat_array are grouped together
spka_combinations = []

# Iterate through all unique combinations of sub_B_array and cat_array
# This will go through all sub_A first, then sub_B, then cat
for a in sub_A_list:
    if sub_B_list:
        for b in sub_B_list:
            if cat_list:  # Check if cat_list is not empty
                for c in cat_list:
                    if add_list:  # Only include d if add_list is not empty
                        for d in add_list:
                            spka_combinations.append((a, b, c, d))
                    else:
                        spka_combinations.append((a, b, c))  # Append without d
            else:  # If cat_list is empty
                if add_list:
                    for d in add_list:
                        spka_combinations.append((a, b, d))  # Append without c
                else:
                    spka_combinations.append((a, b))  # Append only a and b
    else:  # If sub_B_list is empty
        if cat_list:
            for c in cat_list:
                if add_list:
                    for d in add_list:
                        spka_combinations.append((a, c, d))  # Append without b
                else:
                    spka_combinations.append((a, c))  # Append only a and c
        else:  # If both sub_B_list and cat_list are empty
            if add_list:
                for d in add_list:
                    spka_combinations.append((a, d))  # Append only a and d
            else:
                spka_combinations.append((a,))  # Append only a

# Display the combinations
spka_combinations

In [ ]:
# Iterate over each tuple in the list (each separate kinetic profile)
for tuple in spka_combinations:
    
    # Iterate over each number in the tuple (each reagent in an experiment)
    for element in tuple:
        
        # Find the xy position of the vial        
        vial_xy_pos = rack1_commands.get_xy_command(element)
        
        ##### TO DO #####
        
        # Display vial number - must be a two digit integer
        g.bCommand(f"W{element:02}")
        
        # Go to Vial
        g.bCommand(vial_xy_pos[0])
        log_action('test_log.txt', f"Autosampler sent to position {element:02}")
        
        # Needle Down - define this distance somewhere above!
        g.bCommand('Z85')

        # Run the pump
        # This will be a while away...
        # How to make the autosampler idle while this is happening?

        # Needle Up
        g.bCommand('Z120')

        # Log the vial and the amount taken
        # Add a logger...
        print(vial_xy_pos)

        # Create air gap
        # Create a tiny airgap with the pump, will this be necessary?
        
    # Go to the DIM and inject
    g_inject()
    log_action('test_log.txt', "Autosampler sent to DIM to inject")
    
    # Blank the display
    g.bCommand('WBB')

g.bCommand('H')